In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn
seaborn.set(style='ticks')

In [ ]:
import scipy

In [ ]:
import json

In [ ]:
import os

In [ ]:
import mpld3
mpld3.enable_notebook()

In [ ]:
def onset_aligned_intervals(onsets, y):
    
    zeros = np.flatnonzero(librosa.zero_crossings(y))
    
    intervals = librosa.util.frame(onsets, frame_length=2, hop_length=1).T
    
    return zeros[librosa.util.match_events(librosa.frames_to_samples(intervals), zeros)]

In [ ]:
def dftnorm(c):
    
    D = np.fft.rfft(c, axis=0)
    S = np.abs(D)
    energy = np.sum(S**2, axis=0, keepdims=True)**0.5
    energy = librosa.util.normalize(energy, norm=1, axis=1)
    
    Z = np.sum(energy * S, axis=1, keepdims=True)
    
    cinv = np.fft.irfft(D / (1e-8 + Z), axis=0)
    
    if np.iscomplexobj(cinv):
        cinv = cinv.real
        
    return cinv

In [ ]:
def analyze_stem(infile):
    y, sr = librosa.load(infile)
    
    y_harm = librosa.effects.harmonic(y)
    cqgram = librosa.cqt(y_harm, sr=sr, bins_per_octave=12, n_bins=72)
    cqgram = scipy.ndimage.median_filter(cqgram, size=(1, 9))
    cqgram = dftnorm(cqgram)
    
    
    loudness = librosa.logamplitude(librosa.feature.rmse(y)**2)
    
    odf = librosa.onset.onset_strength(y, sr=sr, aggregate=np.median, feature=librosa.cqt, n_bins=72)
    onset_boundaries = librosa.onset.onset_detect(onset_envelope=odf)
    onset_boundaries = librosa.util.fix_frames(onset_boundaries, x_max=cqgram.shape[-1])
    
    cqgram_sync = librosa.feature.sync(cqgram, onset_boundaries, aggregate=np.max)
    loudness_sync = librosa.feature.sync(loudness, onset_boundaries, aggregate=np.mean).ravel()
    
    notes = 24 + np.argmax(librosa.logamplitude(cqgram_sync**2, top_db=30), axis=0)
    
    intervals = onset_aligned_intervals(onset_boundaries, y)
    
    # Convert intervals to time
    times = librosa.samples_to_time(intervals)
    
    threshold = -40
    
    data = zip(list(times[:, 0]),
               list(times[:, 1] - times[:, 0]),
               notes,
               list(loudness_sync),
               [bool(x) for x in list(loudness_sync >= threshold)])
    
    return data

In [ ]:
stems = librosa.util.find_files('~/data/stems/Stevie Wonder - Superstition/', ext='flac')

In [ ]:
for stem_file in stems:
    outfile = os.path.join(os.path.dirname(stem_file), 
                       os.path.extsep.join([os.path.splitext(os.path.basename(stem_file))[0],
                               'json']))
    print outfile
    data = analyze_stem(stem_file)
    with open(outfile, 'w') as f:
        json.dump(data, f, indent=2)